objectives
- find the top hit for each contig in the blast output
- determine what percentage of contigs were classified
- determine what percentage of reads were classified
- compare this to kraken - what is the gain?!

find the top hit for each contig in the blast output

In [ ]:
ENV["LD_LIBRARY_PATH"]

In [ ]:
pkgs = [
    "DataFrames",
    "StatsBase",
    "ProgressMeter",
    "uCSV",
    "FASTX"
]
import Pkg
Pkg.activate(; temp=true)
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
memory=64
threads=max(1, Int(memory//8))
blast_task = "blastn"
db = "nt"

In [ ]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")

In [ ]:
# fasta_files = filter(x -> occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "Expo_pliot1_extracted_hifi_fastqs"), join=true))
# fasta_files = vcat(fasta_files, filter(x -> occursin(r"\.fasta$", x), readdir(joinpath(data_dir, "exposome_data", "extracted1"), join=true)))

# ProgressMeter.@showprogress for fasta_file in fasta_files
#     megablast_result = fasta_file * ".blastn.$(db).megablast.txt"
#     @assert isfile(megablast_result)
#     megablast_result_summary_file = megablast_result * ".summary.tsv"
#     if !isfile(megablast_result_summary_file)
#         hit_strings = readlines(`grep -F "hits found" $(megablast_result)`)
#         hits = map(hit_string -> parse(Int, first(match(r"# (\d+) hits found", hit_string).captures)), hit_strings)
#         query_strings = readlines(`grep -F "# Query:" $(megablast_result)`)
#         query_ids = map(query_string -> first(match(r"# Query: ([^\s]+)", query_string).captures), query_strings)
#         blast_results_summary = DataFrames.DataFrame(;query_ids, hits)
#         uCSV.write(megablast_result_summary_file, blast_results_summary, delim='\t')
#     else
#         blast_results_summary = DataFrames.DataFrame(uCSV.read(megablast_result_summary_file, delim='\t', header=1))
#     end
#     missed_fasta_file = megablast_result * ".missed.fna"
#     missed_ids = Set(blast_results_summary[blast_results_summary[!, "hits"] .== 0, "query_ids"])
#     # @show length(missed_ids)
#     if !isfile(missed_fasta_file)
#         open(missed_fasta_file, "w") do io
#             fastx_io = FASTX.FASTA.Writer(io)
#             for record in FASTX.FASTA.Reader(open(fasta_file))
#                 if FASTX.identifier(record) in missed_ids
#                     write(fastx_io, record)
#                 end
#             end
#             close(fastx_io)
#         end
#     end
#     # @show filesize(missed_fasta_file)
#     target_outfile = missed_fasta_file * ".blastn.$(db).$(blast_task).txt"
#     out_directory = dirname(megablast_result)
#     # @show out_directory
#     if !isfile(target_outfile)
    
#         cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(missed_fasta_file) --blast_db $(db) --task $(blast_task) --out_directory $(dirname(megablast_result)) --threads $(threads)"
#         # @show cmd
#         submission = 
#         `sbatch 
#         --mail-user=cameron.prybol@gmail.com
#         --mail-type=ALL
#         --error=%x-%j.err
#         --output=%x-%j.out
#         --partition=batch
#         --account=mpsnyder
#         --nodes=1
#         --ntasks=1
#         --time=1-00:00:00    
#         --cpus-per-task=$(threads)
#         --mem=$(memory)G
#         --job-name=blastn
#         --wrap $cmd
#         `
#         run(submission)
#     end
# end

In [ ]:
sample_directories = readdir(joinpath(data_dir, "SRA"), join=true)

In [ ]:
# classification_rates_table = DataFrames.DataFrame()
# still need to run 250!!
ProgressMeter.@showprogress for sample_directory in sample_directories
# sample_directory = first(sample_directories)
    # @show sample_directory
    blastn_directory = joinpath(sample_directory, "blastn")
    megablast_result = first(filter(x -> occursin("megablast", x), readdir(blastn_directory, join=true)))
    megablast_result_summary_file = megablast_result * ".summary.tsv"
    try
        if !isfile(megablast_result_summary_file)
            hit_strings = readlines(`grep -F "hits found" $(megablast_result)`)
            hits = map(hit_string -> parse(Int, first(match(r"# (\d+) hits found", hit_string).captures)), hit_strings)
            query_strings = readlines(`grep -F "# Query:" $(megablast_result)`)
            query_ids = map(query_string -> first(match(r"# Query: ([^\s]+)", query_string).captures), query_strings)
            blast_results_summary = DataFrames.DataFrame(;query_ids, hits)
            uCSV.write(megablast_result_summary_file, blast_results_summary, delim='\t')
        else
            blast_results_summary = DataFrames.DataFrame(uCSV.read(megablast_result_summary_file, delim='\t', header=1))
        end
        missed_fasta_file = megablast_result * ".missed.fna"
        fasta_file = joinpath(sample_directory, "megahit", "final.contigs.fastg.gfa.fna")
        missed_ids = Set(blast_results_summary[blast_results_summary[!, "hits"] .== 0, "query_ids"])
        if !isfile(missed_fasta_file) && length(missed_ids) > 0
            open(missed_fasta_file, "w") do io
                fastx_io = FASTX.FASTA.Writer(io)
                for record in FASTX.FASTA.Reader(open(fasta_file))
                    if FASTX.identifier(record) in missed_ids
                        write(fastx_io, record)
                    end
                end
                close(fastx_io)
            end
        end
        if length(missed_ids) > 0
            @assert isfile(missed_fasta_file) && (filesize(missed_fasta_file) > 0)
        end
        target_outfile = missed_fasta_file * ".blastn.$(db).$(blast_task).txt"
        if !isfile(target_outfile) && isfile(missed_fasta_file) && (filesize(missed_fasta_file) > 0)
            cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(missed_fasta_file) --blast_db $(db) --task $(blast_task) --out_directory $(blastn_directory) --threads $(threads)"
            @show cmd
            submission = 
            `sbatch 
            --mail-user=cameron.prybol@gmail.com
            --mail-type=ALL
            --error=%x-%j.err
            --output=%x-%j.out
            --partition=batch
            --account=mpsnyder
            --nodes=1
            --ntasks=1
            --time=1-00:00:00    
            --cpus-per-task=$(threads)
            --mem=$(memory)G
            --job-name=blastn
            --wrap $cmd
            `
            run(submission)
        else
            println("sample $(sample_directory) already run(ning)")
        end
    catch
        println("issue on sample $(sample_directory)")
    end
end

In [ ]:
# sort -k1,1 -k13,13nr -k12,12n  blastout.txt | uniq -k1,1

# sort -u -k1,1 --merge

# bit score

In [ ]:
# query id
# subject id
# subject acc.
# subject acc.ver
# subject title
# query length
# subject length
# q. start
# q. end
# s. start
# s. end
# evalue
# bit score
# alignment length
# % identity
# identical
# mismatches
# subject tax ids

In [ ]:
# sort -k 1 ' < input_file > sorted_file

In [ ]:
 # -k12,12n
# grep -v "^#" /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome-discovery/data/SRA/SRR6399459/blastn/final.contigs.fastg.gfa.fna.blastn.nt.megablast.txt | \
# sort -t $'\t' -k1,1 -k13,13nr | \
# uniq -f 1 | \
# less

determine what percentage of reads were classified

compare this to kraken - what is the gain?!